In [1]:
import pandas as pd
from datetime import date
from random import random, seed

In [2]:
DATOS_FORMULARIO = pd.read_excel('./INPUT/TRANSICIÓN/BASE DE DATOS CICLO 4.xlsx', engine = 'openpyxl', sheet_name='Formulario')
DATOS_M_LLAMADAS = pd.read_excel('./INPUT/TRANSICIÓN/BASE DE DATOS CICLO 4.xlsx', engine = 'openpyxl', sheet_name='Llamada')
DATOS_M_MHORARIO = pd.read_excel('./INPUT/TRANSICIÓN/BASE DE DATOS CICLO 4.xlsx', engine = 'openpyxl', sheet_name='Mismo horario')
DATOS_MAESTRO = pd.concat([DATOS_M_LLAMADAS, DATOS_M_MHORARIO], ignore_index=True)
BD_EST_CICLO_PREVIO = pd.read_excel('./INPUT/TRANSICIÓN/HorariosEstudiantes_C3_2022-09-14.xlsx', engine='openpyxl').sort_values('GRUPO')
BD_EST_CICLO_PREVIO_SIN_HOM = pd.read_excel('./INPUT/TRANSICIÓN/HorariosEstudiantes_C3_2022-09-02_NOHOMS.xlsx', engine='openpyxl')
HOM_ING_C2 = pd.read_excel('./INPUT/TRANSICIÓN/HomsC4.xlsx', engine='openpyxl')['cod'].unique()
DATOS_BALANCEO = pd.read_excel('./INPUT/TRANSICIÓN/BALANCEO_C3-C4.xlsx', engine='openpyxl')['CODIGO'].unique()

CICLO = 4

# Nombres de columnas del formulario
COLUMNA_CODIGOS = 'Código de estudiante'
COLUMNA_CONTINUA_H = """Marque "SÍ" para continuar en el mismo horario del Ciclo 3 - Desarrollo de Software. 
Marque "NO" para seleccionar el horario de disponibilidad de cada uno de los módulos del próximo Ciclo 4 en De..."""
COLUMNA_H_PROG = 'Seleccione por lo menos dos (2) franjas horarias en las cuales tiene disponibilidad de tiempo para atender las clases sincrónicas del módulo de profundización elegido DESARROLLO DE APLICACIONES WE...'
COLUMNA_H_COACH = 'Seleccione por lo menos dos (2) franjas horarias en las cuales tiene disponibilidad de tiempo para atender las clases sincrónicas del módulo COACHING IV / HABILIDADES PERSONALES IV'
COLUMNA_H_ING = 'Seleccione por lo menos dos (2) franjas horarias en las cuales tiene disponibilidad de tiempo para atender las clases sincrónicas del módulo INGLÉS IV'
COLUMNA_RAMA = 'Seleccione el módulo de profundización en Desarrollo de aplicaciones Web (4A) o Desarrollo de aplicaciones Móviles (4B) que cursará en el Ciclo 4.'

HORAS_CONVERSION = {
    'PROGRAMACION': {
        6: 'Jornada de la mañana (6 a.m. a 8 a.m.)',
        8: 'Jornada de la mañana (8 a.m. a 10 a.m.)',
        10: 'Jornada de la mañana (10 a.m. a 12 m.)',
        12: 'Jornada de la tarde (12 m. a 2 p.m.)',
        14: 'Jornada de la tarde (2 p.m. a 4 p.m.)',
        16: 'Jornada de la tarde (4 p.m. a 6 p.m.)',
        18: 'Jornada de la noche (6 p.m. a 8 p.m.)',
        20: 'Jornada de la noche (8 p.m. a 10 p.m.)'
    },
    'INGLES':{
        0: 'HOMOLOGADO',
        6: 'Jornada de la mañana (6 a.m. a 7 a.m.)',
        7: 'Jornada de la mañana (7 a.m. a 8 a.m.)',
        9: 'Jornada de la mañana (9 a.m. a 10 a.m.)',
        11: 'Jornada de la mañana (11 a.m. a 12 m.)',
        12: 'Jornada de la tarde (12 m. a 1 p.m.)',
        13: 'Jornada de la tarde (1 p.m. a 2 p.m.)',
        14: 'Jornada de la tarde (2 p.m. a 3 p.m.)',        
        17: 'Jornada de la tarde (5 p.m. a 6 p.m.)',
        18: 'Jornada de la noche (6 p.m. a 7 p.m.)',
        19: 'Jornada de la noche (7 p.m. a 8 p.m.)',
        20: 'Jornada de la noche (8 p.m. a 9 p.m.)',
        21: 'Jornada de la noche (9 p.m. a 10 p.m.)'
    },
    'COACH': {
        6: 'Jornada de la mañana (6 a.m. a 7 a.m.)',
        7: 'Jornada de la mañana (7 a.m. a 8 a.m.)',
        8: 'Jornada de la mañana (8 a.m. a 9 a.m.)',
        9: 'Jornada de la mañana (9 a.m. a 10 a.m.)',
        10: 'Jornada de la mañana (10 a.m. a 11 a.m.)',
        11: 'Jornada de la mañana (11 a.m. a 12 m.)',
        12: 'Jornada de la tarde (12 m. a 1 p.m.)',
        13: 'Jornada de la tarde (1 p.m. a 2 p.m.)',
        14: 'Jornada de la tarde (2 p.m. a 3 p.m.)',
        15: 'Jornada de la tarde (3 p.m. a 4 p.m.)',
        16: 'Jornada de la tarde (4 p.m. a 5 p.m.)',
        17: 'Jornada de la tarde (5 p.m. a 6 p.m.)',
        18: 'Jornada de la noche (6 p.m. a 7 p.m.)',
        19: 'Jornada de la noche (7 p.m. a 8 p.m.)',
        20: 'Jornada de la noche (8 p.m. a 9 p.m.)',
        21: 'Jornada de la noche (9 p.m. a 10 p.m.)'
    }
}
# Para el asignamiento forzado
FRANJAS_SOLO_INGLES = [
    0, # 6 a 7 am
    1, # 7 a 8 am
    3, # 9 a 10 am
    5, # 11 a 12
    6, # 12 a 1
    7, # 1 a 2
    8, # 2 a 3
    11, # 5 a 6
    12, # 6 a 7
    13, # 7 a 8
    14, # 8 a 9
    15 # 9 a 10
]

In [3]:
FRANJAS_UNIDAS = {
    'MAÑANA': [
        'Jornada de la mañana (6 a.m. a 8 a.m.)',
        'Jornada de la mañana (8 a.m. a 10 a.m.)',
        'Jornada de la mañana (10 a.m. a 12 m.)'
    ],
    'TARDE': [        
        'Jornada de la tarde (12 m. a 2 p.m.)',
        'Jornada de la tarde (2 p.m. a 4 p.m.)',
        'Jornada de la tarde (4 p.m. a 6 p.m.)'
    ],
    'NOCHE': [
        'Jornada de la noche (6 p.m. a 8 p.m.)',
        'Jornada de la noche (8 p.m. a 10 p.m.)'
    ]
}

FRANJAS_GRANULARES = {
    'MAÑANA' : [        
        'Jornada de la mañana (6 a.m. a 7 a.m.)',
        'Jornada de la mañana (7 a.m. a 8 a.m.)',
        'Jornada de la mañana (8 a.m. a 9 a.m.)',
        'Jornada de la mañana (9 a.m. a 10 a.m.)',
        'Jornada de la mañana (10 a.m. a 11 a.m.)',
        'Jornada de la mañana (11 a.m. a 12 m.)'
    ],
    'TARDE': [
        'Jornada de la tarde (12 m. a 1 p.m.)',
        'Jornada de la tarde (1 p.m. a 2 p.m.)',
        'Jornada de la tarde (2 p.m. a 3 p.m.)',
        'Jornada de la tarde (3 p.m. a 4 p.m.)',
        'Jornada de la tarde (4 p.m. a 5 p.m.)',
        'Jornada de la tarde (5 p.m. a 6 p.m.)',
    ],
    'NOCHE': [
        'Jornada de la noche (6 p.m. a 7 p.m.)',
        'Jornada de la noche (7 p.m. a 8 p.m.)',
        'Jornada de la noche (8 p.m. a 9 p.m.)',
        'Jornada de la noche (9 p.m. a 10 p.m.)'
    ]
}
FRANJAS_PROGRAMACION = {}
FRANJAS_INGLES_COACH = {}
TRADUCTOR_PROGRAMACION = {}
TRADUCTOR_INGLES_COACH = {}
i = 0
for jornada in FRANJAS_GRANULARES.values():
    for franja in jornada:        
        FRANJAS_INGLES_COACH[franja] = i
        TRADUCTOR_INGLES_COACH[i] = franja
        i += 1
i = 0
for jornada in FRANJAS_UNIDAS.values():
    for franja in jornada:
        tupla = (i, i+1)
        FRANJAS_PROGRAMACION[franja] = tupla
        TRADUCTOR_PROGRAMACION[tupla] = franja
        i += 2
FRANJAS_INGLES_COACH['HOMOLOGADO'] = -1

In [4]:
def tokenizarAsignado(cadena, separador, conjunto):
    horas = cadena.split(separador)
    rta = []
    for hora in horas:
        if hora in conjunto:
            rta.append(conjunto[hora])
    return rta

def permutar(listaProgramacion, listaIngles, listaCoach):
    for horaInicio, horaFin in listaProgramacion:
        for horaIngles in listaIngles:
            if horaIngles < horaInicio or horaIngles > horaFin:
                for horaCoach in listaCoach:
                    if horaCoach != horaIngles and (horaCoach < horaInicio or horaCoach > horaFin):
                        return (horaInicio, horaFin), horaIngles, horaCoach
    return -1

def todasPermutaciones(listaProgramacion, listaIngles, listaCoach):
    rta = []
    cant = len(listaProgramacion)
    nuevaLista = listaProgramacion.copy()
    while cant > 0:
        res = permutar(nuevaLista, listaIngles, listaCoach)
        if res != -1:
            rta.append(res)
        nuevaLista.pop(0)
        cant -= 1
    return rta

def todasPermutacionesV2(listaProgramacion, listaIngles, listaCoach):
    res = []
    for horaInicio, horaFin in listaProgramacion:
        for horaIngles in listaIngles:
            if horaIngles < horaInicio or horaIngles > horaFin:
                for horaCoach in listaCoach:
                    if horaCoach != horaIngles and (horaCoach < horaInicio or horaCoach > horaFin):
                        res.append(((horaInicio, horaFin), horaIngles, horaCoach))
    return res

def asignarForzadoIngles(listaIngles, horaProg, horaCoach):
    iniProg, finProg = horaProg
    for horaIngles in listaIngles:
        if (horaIngles < iniProg or horaIngles > finProg) and horaIngles != horaCoach:
            return horaIngles
    raise Exception('Error al asignar forzadamente el horario de inglés')

def convertirHora(dicAsignatura, hora):
    rta = dicAsignatura.get(hora, -1)
    if rta == -1:
        raise Exception('Hora no encontrada', hora)
    return rta

def concatenarColumna(col1, col2):
    rta = ''
    if col1 != '':
        rta += col1
    if col2 != '':
        rta += ';' + col2
    return rta + ';'

def generarEstructura():
    return {
        'DOCUMENTO': [],
        'CODIGO': [],
        'NOMBRE': [],
        'APELLIDO': [],
        'TELEFONO_MINTIC': [],
        'TELEFONO_FORMULARIO': [],
        'TELEFONO_SISTEMA_ACADEMICO': [],
        'EMAIL': [],
        'RAMA': [],
        'DISP_PROGRAMACION': [],
        'DISP_INGLES': [],
        'DISP_COACHING': [],
        'TRAB_COLABORATIVO': [],
        'NIVEL_EDUCATIVO_AGRUPADO': [],
        'PRECARGA': [],
        'BALANCEO': [],
        'SOLICITA_CAMBIO': [],
        'CAMBIO_EFECTUADO': []
    }
def ramaAleatoria():
    if random() > 0.70:
        return '4B'
    return '4A'
def identificarRama(rama):
    if rama == 'Ciclo 4A - Desarrollo de aplicaciones Web':
        return '4A'
    return '4B'
def identificarRamaSeg(rama):
    if rama == '4a. Desarrollo WEB':
        return '4A'
    return '4B'

In [5]:
# FASE 1
datosFase1 = generarEstructura()
codigosFormulario = DATOS_FORMULARIO[COLUMNA_CODIGOS].unique()
for index, row in BD_EST_CICLO_PREVIO.iterrows():
    codEst = row['CODIGO_UIS']
    homologoIngles = False
    balanceo = False
    if not(codEst in codigosFormulario): # Para mantener el orden de los grupos
        continue
    if codEst in HOM_ING_C2:
        homologoIngles = True
    dfEst = DATOS_FORMULARIO[DATOS_FORMULARIO[COLUMNA_CODIGOS] == codEst]
    horaDfProg = dfEst[COLUMNA_H_PROG].iloc[0]
    horaDfCoach = dfEst[COLUMNA_H_COACH].iloc[0]
    horaDfIng = dfEst[COLUMNA_H_ING].iloc[0] if not homologoIngles else 'HOMOLOGADO'
    necesitaAnterior = False
    precarga = True
    solicitaCambio = False
    cambioEfectuado = False
    
    if dfEst[COLUMNA_CONTINUA_H].iloc[0] == 'NO': # NO Mantiene los horarios
        precarga = False
        solicitaCambio = True
        cambioEfectuado = True
        horaTempProg = tokenizarAsignado(horaDfProg, ';', FRANJAS_PROGRAMACION)
        horaTempIng = tokenizarAsignado(horaDfIng, ';', FRANJAS_INGLES_COACH)
        horaTempCoach = tokenizarAsignado(horaDfCoach, ';', FRANJAS_INGLES_COACH)
        ### CAMBIADO A V2
        rtaPerm = todasPermutacionesV2(horaTempProg, horaTempIng, horaTempCoach)
        if len(rtaPerm) == 0:
            necesitaAnterior = True
            cambioEfectuado = False
    else:
        necesitaAnterior = True
    if necesitaAnterior:
        horaDfProg = convertirHora(HORAS_CONVERSION['PROGRAMACION'], row['HORA_INICIAL'])
        horaDfIng = convertirHora(HORAS_CONVERSION['INGLES'], row['HORA_INICIAL2']) if not homologoIngles else 'HOMOLOGADO'
        horaDfCoach = convertirHora(HORAS_CONVERSION['COACH'], row['HORA_INICIAL3'])
        if horaDfIng == 'HOMOLOGADO' and not(homologoIngles):
            horaDfIng = BD_EST_CICLO_PREVIO_SIN_HOM[BD_EST_CICLO_PREVIO_SIN_HOM['CODIGO_UIS'] == codEst]['HORA_INICIAL2'].iloc[0]
            horaDfIng = convertirHora(HORAS_CONVERSION['INGLES'], horaDfIng)
            if horaDfIng == 'HOMOLOGADO': # Aún sigue sin horario normal
                tokenProg = tokenizarAsignado(horaDfProg,';', FRANJAS_PROGRAMACION)[0]
                tokenCoach = tokenizarAsignado(horaDfCoach,';', FRANJAS_INGLES_COACH)[0]
                horaDfIng = TRADUCTOR_INGLES_COACH[asignarForzadoIngles(FRANJAS_SOLO_INGLES, tokenProg, tokenCoach)]
        # A estas horas NO se les permuta porque ya deberían de ser válidas. Puesto que eran las del ciclo anterior
        # Hay una muy mínima posibilidad que un homologado haya cambiado de grupo en inglés
    if codEst in DATOS_BALANCEO:
        balanceo = True
        precarga = False
    datosFase1['DOCUMENTO'].append(row['NUMERO_DE_DOCUMENTO'])
    datosFase1['CODIGO'].append(codEst)
    datosFase1['APELLIDO'].append(row['APELLIDOS'])
    datosFase1['NOMBRE'].append(row['NOMBRES'])
    datosFase1['TELEFONO_MINTIC'].append(row['TELEFONO_MINTIC'])
    datosFase1['TELEFONO_FORMULARIO'].append(row['TELEFONO_FORMULARIO'])
    datosFase1['TELEFONO_SISTEMA_ACADEMICO'].append(row['TELEFONO_SISTEMA_ACADEMICO'])
    datosFase1['EMAIL'].append(row['EMAIL'])
    datosFase1['RAMA'].append(identificarRama(dfEst[COLUMNA_RAMA].iloc[0]))
    datosFase1['DISP_PROGRAMACION'].append(horaDfProg)
    datosFase1['DISP_INGLES'].append(horaDfIng)
    datosFase1['DISP_COACHING'].append(horaDfCoach)
    datosFase1['TRAB_COLABORATIVO'].append('NO APLICA')
    datosFase1['NIVEL_EDUCATIVO_AGRUPADO'].append(row['NIVEL_EDUCATIVO'])
    datosFase1['PRECARGA'].append(precarga)
    datosFase1['BALANCEO'].append(balanceo)
    datosFase1['SOLICITA_CAMBIO'].append(solicitaCambio)
    datosFase1['CAMBIO_EFECTUADO'].append(cambioEfectuado)
dfFase1 = pd.DataFrame(datosFase1)

In [6]:
copiaM = DATOS_MAESTRO.copy()
datosFase2 = generarEstructura()
copiaM['HORARIO DESARROLLO WEB O MOVIL PRIORIDAD 1'] = copiaM['HORARIO DESARROLLO WEB O MOVIL PRIORIDAD 1'].fillna('')
copiaM['HORARIO DESARROLLO WEB O MOVIL PRIORIDAD 2'] = copiaM['HORARIO DESARROLLO WEB O MOVIL PRIORIDAD 2'].fillna('')
copiaM['HORARIO INGLES IV PRIORIDAD 1'] = copiaM['HORARIO INGLES IV PRIORIDAD 1'].fillna('')
copiaM['HORARIO INGLES IV PRIORIDAD 2'] = copiaM['HORARIO INGLES IV PRIORIDAD 2'].fillna('')
copiaM['HORARIO HAB. PERSONALES IV PRIORIDAD 1'] = copiaM['HORARIO HAB. PERSONALES IV PRIORIDAD 1'].fillna('')
copiaM['HORARIO HAB. PERSONALES IV PRIORIDAD 2'] = copiaM['HORARIO HAB. PERSONALES IV PRIORIDAD 2'].fillna('')
# Especulación
copiaM['¿CICLO 4a o 4b?'] = copiaM['¿CICLO 4a o 4b?'].fillna('')
for index, row in copiaM.iterrows():
    codEst = row['CODIGO_UIS']
    if codEst in datosFase1['CODIGO']: # Para evitar tocar a los precargados
        continue
    necesitaAnterior = False
    homologoIngles = False
    precarga = True
    solicitaCambio = False
    cambioEfectuado = False
    balanceo = False
    
    if codEst in HOM_ING_C2:
        homologoIngles = True
    
    horaDfProg = concatenarColumna(row['HORARIO DESARROLLO WEB O MOVIL PRIORIDAD 1'], row['HORARIO DESARROLLO WEB O MOVIL PRIORIDAD 2'])
    horaDfIng = concatenarColumna(row['HORARIO INGLES IV PRIORIDAD 1'], row['HORARIO INGLES IV PRIORIDAD 2']) if not homologoIngles else 'HOMOLOGADO'
    horaDfCoach = concatenarColumna(row['HORARIO HAB. PERSONALES IV PRIORIDAD 1'], row['HORARIO HAB. PERSONALES IV PRIORIDAD 2'])
    datosEstBdAnterior = BD_EST_CICLO_PREVIO[BD_EST_CICLO_PREVIO['CODIGO_UIS'] == codEst]
    
    if horaDfProg != ';' and horaDfIng != ';' and horaDfCoach != ';':
        precarga = False
        solicitaCambio = True
        cambioEfectuado = True
        lProg = tokenizarAsignado(horaDfProg, ';', FRANJAS_PROGRAMACION)
        lIng = tokenizarAsignado(horaDfIng, ';', FRANJAS_INGLES_COACH)
        lCoach = tokenizarAsignado(horaDfCoach, ';', FRANJAS_INGLES_COACH)
        ### CAMBIADO A V2
        perms = todasPermutacionesV2(lProg, lIng, lCoach) 
        if len(perms) == 0:
            necesitaAnterior = True
            cambioEfectuado = False
    else:
        necesitaAnterior = True
    
    if necesitaAnterior:
        horaDfProg = convertirHora(HORAS_CONVERSION['PROGRAMACION'], datosEstBdAnterior['HORA_INICIAL'].iloc[0])
        horaDfIng = convertirHora(HORAS_CONVERSION['INGLES'], datosEstBdAnterior['HORA_INICIAL2'].iloc[0]) if not homologoIngles else 'HOMOLOGADO'
        horaDfCoach = convertirHora(HORAS_CONVERSION['COACH'], datosEstBdAnterior['HORA_INICIAL3'].iloc[0])
        if horaDfIng == 'HOMOLOGADO' and not(homologoIngles):
            horaDfIng = BD_EST_CICLO_PREVIO_SIN_HOM[BD_EST_CICLO_PREVIO_SIN_HOM['CODIGO_UIS'] == codEst]['HORA_INICIAL2'].iloc[0]
            horaDfIng = convertirHora(HORAS_CONVERSION['INGLES'], horaDfIng)
            if horaDfIng == 'HOMOLOGADO': # Aún sigue sin horario normal
                tokenProg = tokenizarAsignado(horaDfProg,';', FRANJAS_PROGRAMACION)[0]
                tokenCoach = tokenizarAsignado(horaDfCoach,';', FRANJAS_INGLES_COACH)[0]
                horaDfIng = TRADUCTOR_INGLES_COACH[asignarForzadoIngles(FRANJAS_SOLO_INGLES, tokenProg, tokenCoach)]
    if codEst in DATOS_BALANCEO:
        balanceo = True
        precarga = False
    datosFase2['DOCUMENTO'].append(row['DOCUMENTO_IDENTIFICACION '])
    datosFase2['CODIGO'].append(codEst)
    datosFase2['APELLIDO'].append(row['APELLIDOS'])
    datosFase2['NOMBRE'].append(row['NOMBRES'])
    datosFase2['TELEFONO_MINTIC'].append(datosEstBdAnterior['TELEFONO_MINTIC'].iloc[0])
    datosFase2['TELEFONO_FORMULARIO'].append(datosEstBdAnterior['TELEFONO_FORMULARIO'].iloc[0])
    datosFase2['TELEFONO_SISTEMA_ACADEMICO'].append(datosEstBdAnterior['TELEFONO_SISTEMA_ACADEMICO'].iloc[0])
    datosFase2['EMAIL'].append(row['EMAIL'])    
    datosFase2['RAMA'].append(identificarRamaSeg(row['¿CICLO 4a o 4b?']) if row['¿CICLO 4a o 4b?'] != '' else ramaAleatoria()) #Cambiar el nombre de la columna por el correcto
    datosFase2['DISP_PROGRAMACION'].append(horaDfProg)
    datosFase2['DISP_INGLES'].append(horaDfIng)
    datosFase2['DISP_COACHING'].append(horaDfCoach)
    datosFase2['TRAB_COLABORATIVO'].append('NO APLICA')
    datosFase2['NIVEL_EDUCATIVO_AGRUPADO'].append(datosEstBdAnterior['NIVEL_EDUCATIVO'].iloc[0])
    datosFase2['PRECARGA'].append(precarga)
    datosFase2['BALANCEO'].append(balanceo)
    datosFase2['SOLICITA_CAMBIO'].append(solicitaCambio)
    datosFase2['CAMBIO_EFECTUADO'].append(cambioEfectuado)
dfFase2 = pd.DataFrame(datosFase2)

In [7]:
sufijo = 'CICLO_' + str(CICLO) + '_' + str(date.today())
dfGeneral = pd.concat([dfFase1, dfFase2], ignore_index=True)
dfGeneral.to_excel('./OUTPUT/TRANSICIÓN/CONSOLIDADO_TODOS_'+sufijo+'.xlsx', index = False)
print (len(dfGeneral))
dfPrecarga = dfGeneral[dfGeneral['PRECARGA'] == True]
dfCambios = dfGeneral[(dfGeneral['PRECARGA'] == False) & (dfGeneral['BALANCEO'] == False)]
dfBalanceo = dfGeneral[dfGeneral['BALANCEO'] == True]
dfPrecarga.to_excel('./OUTPUT/TRANSICIÓN/CONSOLIDADO_PRECARGA_'+sufijo+'.xlsx', index = False)
dfCambios.to_excel('./OUTPUT/TRANSICIÓN/CONSOLIDADO_EST_'+sufijo+'.xlsx', index = False)
dfBalanceo.to_excel('./OUTPUT/TRANSICIÓN/CONSOLIDADO_BALANCEO_'+sufijo+'.xlsx', index = False)

3585


# DISTRIBUCIÓN DE GRUPOS

In [8]:
def iniciarContadores(jornadas):
    dic = {}
    for jornada in jornadas:
        dic[jornada] = 0
    return dic

In [9]:
CONTEOS = {
    'PROGRAMACIÓN': iniciarContadores(FRANJAS_PROGRAMACION.keys()),
    'INGLÉS': iniciarContadores(FRANJAS_INGLES_COACH.keys()),
    'COACHING': iniciarContadores(FRANJAS_INGLES_COACH.keys()),
}
for index, row in dfGeneral.iterrows():
    a = row['DISP_PROGRAMACION'].split(';')
    b = row['DISP_INGLES'].split(';')
    c = row['DISP_COACHING'].split(';')
    jornadasEstudiante = {
        'PROGRAMACIÓN': a,
        'INGLÉS': b,
        'COACHING': c
    }
    for jornada in jornadasEstudiante:
        for horario in jornadasEstudiante[jornada]:
            if horario == '':
                continue
            if horario in CONTEOS[jornada]:
                CONTEOS[jornada][horario] += 1
            else:
                raise Exception(horario, jornada)

In [10]:
for tema in CONTEOS:
    suma = 0
    print ('========')
    print ('Horarios de', tema)
    for horario in CONTEOS[tema]:
        suma += CONTEOS[tema][horario]
    for horario in CONTEOS[tema]:
        total = CONTEOS[tema][horario]
        msg = 'Franja: {franja}, cantidad: {cantidad}, porcentaje: {porcentaje:.2f}'
        print (msg.format(franja = horario, cantidad = total, porcentaje = total/suma*100))
    print ('Total elecciones: ', suma)

Horarios de PROGRAMACIÓN
Franja: Jornada de la mañana (6 a.m. a 8 a.m.), cantidad: 387, porcentaje: 9.96
Franja: Jornada de la mañana (8 a.m. a 10 a.m.), cantidad: 437, porcentaje: 11.25
Franja: Jornada de la mañana (10 a.m. a 12 m.), cantidad: 229, porcentaje: 5.90
Franja: Jornada de la tarde (12 m. a 2 p.m.), cantidad: 132, porcentaje: 3.40
Franja: Jornada de la tarde (2 p.m. a 4 p.m.), cantidad: 219, porcentaje: 5.64
Franja: Jornada de la tarde (4 p.m. a 6 p.m.), cantidad: 250, porcentaje: 6.44
Franja: Jornada de la noche (6 p.m. a 8 p.m.), cantidad: 1014, porcentaje: 26.11
Franja: Jornada de la noche (8 p.m. a 10 p.m.), cantidad: 1216, porcentaje: 31.31
Total elecciones:  3884
Horarios de INGLÉS
Franja: Jornada de la mañana (6 a.m. a 7 a.m.), cantidad: 253, porcentaje: 6.57
Franja: Jornada de la mañana (7 a.m. a 8 a.m.), cantidad: 266, porcentaje: 6.91
Franja: Jornada de la mañana (8 a.m. a 9 a.m.), cantidad: 0, porcentaje: 0.00
Franja: Jornada de la mañana (9 a.m. a 10 a.m.), cant